### CaseHOLD labeling - Nova models

In this notebook, we run a labeling procedure on the test set of the CaseHOLD dataset. The dataset uses a "citing prompt", and contains five multiple choice answer options. We ask the LLM to read over the prompt and options, then select the best one.

We test three different LLMs from Amazon. We record the responses for each model for further analysis elsewhere.

In [5]:
import pandas as pd
from datasets import load_dataset
from langchain_aws import ChatBedrockConverse
from boto3 import client
from botocore.config import Config
from bedrock_utils import BedrockBatchProcessor
from config import S3_BUCKET, ROLE_ARN

config = Config(read_timeout=1000)

client = client(service_name='bedrock-runtime',
                      config=config, region_name="us-east-1")

ds = load_dataset("casehold/casehold", "all")

In [3]:
def create_prompt(r):
    return """
    Task: Legal Holding Identification
    
    Context: You are analyzing a legal text to identify the most appropriate legal holding. A legal holding is the court's determination of a matter of law based on the facts of a particular case.
    
    Input Text: "{citing_prompt}"

    Question: Based on the legal context above, which of the following holdings best completes the text where the <HOLDING> tag appears? Consider:
    - The specific legal issue being discussed
    - The logical flow of the legal argument 
    - The precedential value implied by the context

    Options:
    A: {holding_0}
    B: {holding_1}
    C: {holding_2}
    D: {holding_3}
    E: {holding_4}

    Instructions:
    1. Analyze the context and legal reasoning in the input text
    2. Consider how each option would fit within the legal argument
    3. Evaluate which option best maintains the logical flow. Explain your reasoning first, formatted like this <reasoning> reasoning </reasoning>
    4. Provide your final answer in the format: ANSWER: X (where X is A, B, C, D, or E)""".format(
        citing_prompt = r['citing_prompt'], holding_0 = r['holding_0'], holding_1 = r['holding_1'], 
        holding_2 = r['holding_2'], holding_3 = r['holding_3'], holding_4 = r['holding_4'])

In [4]:
test = ds['test'].to_pandas()
test['label'] = test['label'].astype(float)
test['final_prompt'] = test.apply(create_prompt, axis = 1)

In [6]:
batch = BedrockBatchProcessor(bucket = S3_BUCKET, role_arn=ROLE_ARN, model_id='amazon.nova-micro-v1:0', output_path='casehold/outputs/nova_micro', key_prefix = 'casehold/inputs/nova_micro/nova_micro')

In [ ]:
batch.process_data(test['

In [ ]:
test['response_nova_micro'] = ''
test['response_nova_lite'] = ''
test['response_nova_pro'] = ''

In [ ]:
llm_nova_micro = ChatBedrockConverse(model="amazon.nova-micro-v1:0", region_name="us-east-1", temperature = 0, client = client)
llm_nova_lite = ChatBedrockConverse(model="amazon.nova-lite-v1:0", region_name="us-east-1", temperature = 0, client = client)
llm_nova_pro = ChatBedrockConverse(model="amazon.nova-pro-v1:0", region_name="us-east-1", temperature = 0, client = client)

In [ ]:
for i in range(0, test.shape[0]):
    
    print(i)
    row = test.loc[i]
    messages = [
        ("user", row['final_prompt'])
    ]

    a = llm_nova_micro.invoke(messages)
    test.loc[i, 'response_nova_micro'] = a.content
    
    b = llm_nova_lite.invoke(messages)
    test.loc[i, 'response_nova_lite'] = b.content

    c = llm_nova_pro.invoke(messages)
    test.loc[i, 'response_nova_pro'] = c.content

In [ ]:
test.to_csv('casehold_test_nova_labels.csv')